# Exploring the Spider Dataset

This notebook explores the Spider text-to-SQL dataset to understand:
1. Dataset structure (questions, SQL queries, schemas)
2. What data we'll use for retrieval (schemas, examples, similar questions)
3. How to format this information for our RAG system

In [1]:
import json
import os
from pathlib import Path
import pandas as pd
import sqlite3

# Set path to your Spider data
SPIDER_DATA_PATH = Path(r'F:\text2sql\spider_data')

print(f"Spider data location: {SPIDER_DATA_PATH}")
print(f"Exists: {SPIDER_DATA_PATH.exists()}")

Spider data location: F:\text2sql\spider_data
Exists: True


## 1. Load Training Data

Load `train_spider.json` to see example questions and SQL queries

In [2]:
# Load training data
with open(SPIDER_DATA_PATH / 'train_spider.json', 'r') as f:
    train_data = json.load(f)

print(f"Number of training examples: {len(train_data)}")
print(f"\nFirst example:")
print(json.dumps(train_data[0], indent=2))

Number of training examples: 7000

First example:
{
  "db_id": "department_management",
  "query": "SELECT count(*) FROM head WHERE age  >  56",
  "query_toks": [
    "SELECT",
    "count",
    "(",
    "*",
    ")",
    "FROM",
    "head",
    "WHERE",
    "age",
    ">",
    "56"
  ],
  "query_toks_no_value": [
    "select",
    "count",
    "(",
    "*",
    ")",
    "from",
    "head",
    "where",
    "age",
    ">",
    "value"
  ],
  "question": "How many heads of the departments are older than 56 ?",
  "question_toks": [
    "How",
    "many",
    "heads",
    "of",
    "the",
    "departments",
    "are",
    "older",
    "than",
    "56",
    "?"
  ],
  "sql": {
    "from": {
      "table_units": [
        [
          "table_unit",
          1
        ]
      ],
      "conds": []
    },
    "select": [
      false,
      [
        [
          3,
          [
            0,
            [
              0,
              0,
              false
            ],
            null
     

## 2. Examine Example Structure

Look at a few examples to understand the data format

In [3]:
# Look at first 5 examples
print("First 5 training examples:\n")
for i, example in enumerate(train_data[:5]):
    print(f"Example {i+1}:")
    print(f"  Database: {example['db_id']}")
    print(f"  Question: {example['question']}")
    print(f"  SQL: {example['query']}")
    print()

First 5 training examples:

Example 1:
  Database: department_management
  Question: How many heads of the departments are older than 56 ?
  SQL: SELECT count(*) FROM head WHERE age  >  56

Example 2:
  Database: department_management
  Question: List the name, born state and age of the heads of departments ordered by age.
  SQL: SELECT name ,  born_state ,  age FROM head ORDER BY age

Example 3:
  Database: department_management
  Question: List the creation year, name and budget of each department.
  SQL: SELECT creation ,  name ,  budget_in_billions FROM department

Example 4:
  Database: department_management
  Question: What are the maximum and minimum budget of the departments?
  SQL: SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department

Example 5:
  Database: department_management
  Question: What is the average number of employees of the departments whose rank is between 10 and 15?
  SQL: SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AN

## 3. Load Schema Information

Load `tables.json` to see database schemas (tables, columns, types)

In [4]:
# Load schema information
with open(SPIDER_DATA_PATH / 'tables.json', 'r') as f:
    tables_data = json.load(f)

print(f"Number of databases: {len(tables_data)}")
print(f"\nFirst database schema:")
print(json.dumps(tables_data[0], indent=2))

Number of databases: 166

First database schema:
{
  "column_names": [
    [
      -1,
      "*"
    ],
    [
      0,
      "perpetrator id"
    ],
    [
      0,
      "people id"
    ],
    [
      0,
      "date"
    ],
    [
      0,
      "year"
    ],
    [
      0,
      "location"
    ],
    [
      0,
      "country"
    ],
    [
      0,
      "killed"
    ],
    [
      0,
      "injured"
    ],
    [
      1,
      "people id"
    ],
    [
      1,
      "name"
    ],
    [
      1,
      "height"
    ],
    [
      1,
      "weight"
    ],
    [
      1,
      "home town"
    ]
  ],
  "column_names_original": [
    [
      -1,
      "*"
    ],
    [
      0,
      "Perpetrator_ID"
    ],
    [
      0,
      "People_ID"
    ],
    [
      0,
      "Date"
    ],
    [
      0,
      "Year"
    ],
    [
      0,
      "Location"
    ],
    [
      0,
      "Country"
    ],
    [
      0,
      "Killed"
    ],
    [
      0,
      "Injured"
    ],
    [
      1,
      "Peopl

## 4. Examine a Specific Database Schema

Look at the schema structure in detail

In [5]:
# Pick a database and examine its schema
example_db = tables_data[0]

print(f"Database ID: {example_db['db_id']}")
print(f"\nTable names: {example_db['table_names_original']}")
print(f"\nColumn names (first 10):")
for i, col in enumerate(example_db['column_names_original'][:10]):
    table_idx, col_name = col
    if table_idx >= 0:
        table_name = example_db['table_names_original'][table_idx]
        print(f"  {table_name}.{col_name}")
    else:
        print(f"  {col_name}")

print(f"\nColumn types (first 10): {example_db['column_types'][:10]}")

Database ID: perpetrator

Table names: ['perpetrator', 'people']

Column names (first 10):
  *
  perpetrator.Perpetrator_ID
  perpetrator.People_ID
  perpetrator.Date
  perpetrator.Year
  perpetrator.Location
  perpetrator.Country
  perpetrator.Killed
  perpetrator.Injured
  people.People_ID

Column types (first 10): ['text', 'number', 'number', 'text', 'number', 'text', 'text', 'number', 'number', 'number']


## 5. Understand Foreign Keys

Foreign keys show relationships between tables

In [6]:
# Look at foreign keys
if 'foreign_keys' in example_db:
    print(f"Foreign keys in {example_db['db_id']}:")
    for fk in example_db['foreign_keys']:
        from_col_idx, to_col_idx = fk
        from_table_idx, from_col = example_db['column_names_original'][from_col_idx]
        to_table_idx, to_col = example_db['column_names_original'][to_col_idx]
        from_table = example_db['table_names_original'][from_table_idx]
        to_table = example_db['table_names_original'][to_table_idx]
        print(f"  {from_table}.{from_col} -> {to_table}.{to_col}")

Foreign keys in perpetrator:
  perpetrator.People_ID -> people.People_ID


## 6. Load Development Set

This is what we'll use for evaluation

In [7]:
# Load dev set
with open(SPIDER_DATA_PATH / 'dev.json', 'r') as f:
    dev_data = json.load(f)

print(f"Number of dev examples: {len(dev_data)}")
print(f"\nFirst dev example:")
print(json.dumps(dev_data[0], indent=2))

Number of dev examples: 1034

First dev example:
{
  "db_id": "concert_singer",
  "query": "SELECT count(*) FROM singer",
  "query_toks": [
    "SELECT",
    "count",
    "(",
    "*",
    ")",
    "FROM",
    "singer"
  ],
  "query_toks_no_value": [
    "select",
    "count",
    "(",
    "*",
    ")",
    "from",
    "singer"
  ],
  "question": "How many singers do we have?",
  "question_toks": [
    "How",
    "many",
    "singers",
    "do",
    "we",
    "have",
    "?"
  ],
  "sql": {
    "from": {
      "table_units": [
        [
          "table_unit",
          1
        ]
      ],
      "conds": []
    },
    "select": [
      false,
      [
        [
          3,
          [
            0,
            [
              0,
              0,
              false
            ],
            null
          ]
        ]
      ]
    ],
    "where": [],
    "groupBy": [],
    "having": [],
    "orderBy": [],
    "limit": null,
    "intersect": null,
    "union": null,
    "except": null


## 7. Database Statistics

Understand the distribution of questions across databases

In [8]:
# Count questions per database in training set
from collections import Counter

train_db_counts = Counter([ex['db_id'] for ex in train_data])
dev_db_counts = Counter([ex['db_id'] for ex in dev_data])

print(f"Number of unique databases in training: {len(train_db_counts)}")
print(f"Number of unique databases in dev: {len(dev_db_counts)}")

print(f"\nTop 10 databases by training examples:")
for db_id, count in train_db_counts.most_common(10):
    print(f"  {db_id}: {count} examples")

Number of unique databases in training: 140
Number of unique databases in dev: 20

Top 10 databases by training examples:
  college_2: 170 examples
  college_1: 164 examples
  hr_1: 124 examples
  store_1: 112 examples
  soccer_2: 106 examples
  bike_1: 104 examples
  music_1: 100 examples
  hospital_1: 100 examples
  music_2: 100 examples
  dorm_1: 100 examples


## 8. Test Database Connection

Try connecting to one of the SQLite databases

In [9]:
# Connect to a database and examine it
test_db_id = train_data[0]['db_id']
db_path = SPIDER_DATA_PATH / 'database' / test_db_id / f'{test_db_id}.sqlite'

print(f"Testing connection to: {db_path}")
print(f"Database exists: {db_path.exists()}")

if db_path.exists():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get list of tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"\nTables in {test_db_id}:")
    for table in tables:
        print(f"  - {table[0]}")
    
    conn.close()
else:
    print("Database file not found!")

Testing connection to: F:\text2sql\spider_data\database\department_management\department_management.sqlite
Database exists: True

Tables in department_management:
  - department
  - head
  - management


## 9. Execute a Sample Query

Test executing a SQL query from the dataset

In [10]:
# Execute the first training example
example = train_data[0]
db_path = SPIDER_DATA_PATH / 'database' / example['db_id'] / f"{example['db_id']}.sqlite"

print(f"Question: {example['question']}")
print(f"SQL: {example['query']}")

if db_path.exists():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    try:
        cursor.execute(example['query'])
        results = cursor.fetchall()
        print(f"\nResults: {results[:5]}")
        print(f"Total rows: {len(results)}")
    except Exception as e:
        print(f"Error executing query: {e}")
    
    conn.close()

Question: How many heads of the departments are older than 56 ?
SQL: SELECT count(*) FROM head WHERE age  >  56

Results: [(5,)]
Total rows: 1


## 10. Summary: What We'll Use for RAG

Based on this exploration, here's what we'll retrieve:

1. **Schema Information** (from `tables.json`):
   - Table names
   - Column names and types
   - Foreign key relationships

2. **Example Queries** (from `train_spider.json`):
   - Natural language questions
   - Corresponding SQL queries
   - Database context

3. **Similar Questions**:
   - Questions from the same database
   - Questions with similar SQL patterns

Next steps:
- Build embeddings for questions and schemas
- Create FAISS index
- Implement retrieval function

In [11]:
# Print summary statistics
print("=" * 50)
print("SPIDER DATASET SUMMARY")
print("=" * 50)
print(f"Training examples: {len(train_data)}")
print(f"Dev examples: {len(dev_data)}")
print(f"Number of databases: {len(tables_data)}")
print(f"Unique databases in training: {len(train_db_counts)}")
print(f"Unique databases in dev: {len(dev_db_counts)}")
print("=" * 50)

SPIDER DATASET SUMMARY
Training examples: 7000
Dev examples: 1034
Number of databases: 166
Unique databases in training: 140
Unique databases in dev: 20
